# Getting started with liquid handling on a Hamilton STAR(let)

In this notebook, you will learn how to use PyLabRobot to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyLabRobot and the USB driver as described in [the installation guide](/user_guide/installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:


## Setting up a connection with the robot

Start by importing the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pylabrobot.liquid_handling.backends.STAR` backend.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import EVO

In addition, import the {class}`~pylabrobot.resources.hamilton.STARLetDeck`, which represents the deck of the Hamilton STAR.

In [3]:
from pylabrobot.resources.tecan import EVO200Deck

Create a new liquid handler using `STAR` as its backend.

In [4]:
backend = EVO()
lh = LiquidHandler(backend=backend, deck=EVO200Deck())


# backend.num_channels
# backend.diti_count = 96 # need to add the number of tips to the backend. TODO should be done automactically

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [5]:


await lh.setup()



Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02W1\x80\x00')
{'module': 'W1', 'data': []}
Raw Response: bytearray(b'\x02W1\x80\x00')
Raw Response: bytearray(b'\x02C1\x80\x00')
Raw Response: bytearray(b'\x02C1\x80\x00')
Raw Response: bytearray(b'\x02C1\x80\x00')
RoMa connected!
Raw Response: bytearray(b'\x02C1\x803588\x00')
cur_x RoMa() 3588 <class 'int'>
x RoMa() 9000 <class 'int'>
pos RoMa() 3588 <class 'int'>
Raw Response: bytearray(b'\x02C1\x80\x00')
Raw Response: bytearray(b'\x02C1\x80\x00')
Raw Response: bytearray(b'\x02C1\x809000\x00')
MCA connected!
Raw Response: bytearray(b'\x02W1\x80\x00')
Raw Response: bytearray(b'\x02W1\x80\x00')
Raw Response: bytearray(b'\x02W1\x80\x00')
Raw Response: bytearray(b'\x02W1\x80\x00')
MCA parked safely.
Raw Response: bytearray(b'\x02C5\x80\x00')
LiHa connected!
Raw Response: bytearray(b'\x02C5\x808\x00')
Raw Response: bytearray(b'\x02C5\x8013559\x00')
Raw Response: bytearray(b'\

In [6]:
diti_cones = True
## if on LiHa you are using DITI cones then set
if diti_cones:
    diti_count = 8
    lh.diti_count = diti_count
else:
  ## If usinng fixed tips then set
  from pylabrobot.resources.tecan.tip_creators import standard_fixed_tip
  for i in range(lh.backend.num_channels):
    lh.head[i]._tip = standard_fixed_tip()

In [7]:
backend.roma_connected

True

In [8]:
backend.liha_connected

True

In [9]:
backend.mca_connected # not acting. some error in EVO.py try to run in debug mode and see where the ROMA and LiHa activate
## WHY DOES IT NOT ENTER INTO THE LOOP

True

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyLabRobot package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.hamilton.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.hamilton.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cor_96_wellplate_360ul_Fb` wells
- {class}`~pylabrobot.resources.hamilton.tip_racks.HTF` tips

In [10]:
from pylabrobot.resources import (
  MP_3Pos,
  MP_4Pos_flat,
  Washstation_2Grid_Trough_DiTi, # selected based on partnumer 10650037 from page 194 https://www.tecan.com/hubfs/402249_en_V1_0%20OM%20Freedom%20EVOlyzer-3%20for%20GP%20Use-1.pdf

  # added labware/tipracks(aka tipboxes)
  DiTi_1000ul_SBS_LiHa, # for testing maybe similar to specs of flat carrie. page 187
  DiTi_1000ul_LiHa, # test how that works
  DiTi_1000ul_Filter_LiHa, # test how that works
  DiTi_50ul_MCA384,
  DiTi_500ul_SBS_MCA96,
  DiTi_200ul_SBS_LiHa, # free haning tips

  Microplate_96_Well,

  # added tip_carrier
  DiTi_SBS_4_Pos_MCA96,
  DiTi_3Pos, # haning tips

)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [11]:
## QUESTION TO ANSWAR:
# Can i place diti tip racks on carriers?

# Can i run diti tip racks
# can i run the MCA head
# can i pick up tips using the MCA head?

# can i wash in the washstation.
# Is it the correct wash station??

# Can u use LLD?
# can i turn off LLD?

# can i flush / wash liquid



In [42]:
# tip_car = MP_4Pos_flat(name="tip carrier")
# tip_car[0] = Microplate_96_Well(name="micro_plate_tip_naigbor")

tip_car_hang = DiTi_3Pos(name="tip carrier hang")
tip_car_hang[0] = DiTi_200ul_SBS_LiHa(name="tiprack_1") # this gives problems. probably because it has not been tesetd enough
tip_car_hang[1] = DiTi_1000ul_Filter_LiHa(name="tiprack_2")

In [ ]:
# tip_car[1] = DiTi_1000ul_SBS_LiHa(name="tips_02")
# lh.deck.unassign_child_resource(tip_car_hang)

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [33]:
# lh.deck.assign_child_resource(tip_car, rails=46)

# lh.deck.assign_child_resource(tip_car_hang, rails=34, reassign=True) # real life position
lh.deck.assign_child_resource(tip_car_hang, rails=7, reassign=True) # fake test position

# generate plate carrier

In [15]:
plt_car = MP_3Pos(name="plate carrier")
plt_car[0] = Microplate_96_Well(name="plate_01")
lh.deck.assign_child_resource(plt_car, rails=3) # other locations of the same carrier 9, 15, 21

Repeat this for the plates.

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [105]:
lh.summary()

Rail     Resource                   Type                Coordinates (mm)
(1)  ├── wash_station               TecanWashStation    Coordinate(087.500, -20.300, 000.000)
     │   ├── wash_clean_deep        Trash               Coordinate(099.700, 086.400, 000.000)
     │   ├── wash_waste             Trash               Coordinate(098.500, 160.400, 000.000)
     │   ├── wash_clean_shallow     Trash               Coordinate(099.700, 261.400, 000.000)
     │
(3)  ├── plate carrier              TecanPlateCarrier   Coordinate(138.000, 053.700, 000.000)
     │   ├── plate_01               TecanPlate          Coordinate(160.500, 142.300, 062.500)
     │   ├── <empty>
     │   ├── <empty>
     │
(7)  ├── tip carrier hang           TecanTipCarrier     Coordinate(238.000, -04.300, 000.000)
     │   ├── tiprack_1              TecanTipRack        Coordinate(265.900, 140.600, -00.500)
     │   ├── tiprack_2              TecanTipRack        Coordinate(263.500, 241.900, 037.100)
     │   ├── <empty>
    

In [17]:
# backend.setup_finished = True
lh.setup_finished

True

In [18]:
lh.head

{0: TipTracker(Channel 0, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 1: TipTracker(Channel 1, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 2: TipTracker(Channel 2, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 3: TipTracker(Channel 3, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 4: TipTracker(Channel 4, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 5: TipTracker(Channel 5, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 6: TipTracker(Channel 6, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 7: TipTracker(Channel 7, is_disabled=False, has_tip=False tip=None, pending_tip=None)}

In [66]:
tiprack = lh.deck.get_resource("tiprack_1")

In [101]:
# lh.num_channels
# await lh.move_channel_x(channel=1, x=100)

In [ ]:
from pylabrobot.resources import Coordinate
from pylabrobot.liquid_handling.standard import Pickup
tiprack = lh.deck.get_resource("tiprack_1")  # Get the tip rack instance
tip = tiprack.get_tip("A1")  # Get the tip identificer
# Get the TipSpot
tip_spot = tiprack["A1"]

# Create a Pickup operation with an offset
pickup_op = Pickup(
    resource=tip_spot,
    offset=Coordinate(50, 50, 0),  # Apply the X, Y, Z offset
    tip=tip
)
print(pickup_op)

# Run the pickup with the offset
await lh.pick_up_tips(tip_spot, ops=pickup_op, offsets=[Coordinate(-15, -15, 0)])
# await backend.pick_up_tips([pickup_op], use_channels=[0])

# GREAT I CAN NOW PICK UP A TIP!!!

Pickup(resource=[TipSpot(name=tiprack_1_tipspot_0_0, location=Coordinate(010.100, 070.400, -05.000), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)], offset=Coordinate(x=50, y=50, z=0), tip=TecanTip(has_filter=False, total_tip_length=21.0, maximal_volume=220.0, fitting_depth=0))
Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02C5\x80-147\x00')
Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02C5\x80\x00')
Raw Response: bytearray(b'\x02C5\x80\x00')


In [ ]:
### NEXT STEP DEFINE A WASTE STATION AND A WASH STATION
# THEN TRY TO DISCARD TIP INTO WASTE STATION
multi_station = Washstation_2Grid_Trough_DiTi(name="washstation")

lh.deck.assign_child_resource(multi_station, rails=15) # Fake position

In [106]:
await lh.drop_tips(multi_station)


KeyError: 8

In [ ]:
lh.head()

{0: TipTracker(Channel 0, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 1: TipTracker(Channel 1, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 2: TipTracker(Channel 2, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 3: TipTracker(Channel 3, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 4: TipTracker(Channel 4, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 5: TipTracker(Channel 5, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 6: TipTracker(Channel 6, is_disabled=False, has_tip=False tip=None, pending_tip=None),
 7: TipTracker(Channel 7, is_disabled=False, has_tip=False tip=None, pending_tip=None)}

In [62]:
# tiprack = lh.deck.get_resource("tiprack_1")

# await lh.pick_up_tips(tiprack["A1:H1"], use_channels=list(range(backend.num_channels)))
# await lh.pick_up_tips(tiprack["A1"], use_channels=[1])

await lh.pick_up_tips(
    [tiprack["A1"]],  # Wrap in a list
    use_channels=[0]  # Use only one channel
)


TypeError: Resources must be `TipSpot`s, got [[TipSpot(name=tiprack_2_tipspot_0_0, location=Coordinate(007.700, 071.700, 032.600), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot)]]

## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [21]:
tiprack = lh.deck.get_resource("tiprack_1")
await lh.pick_up_tips(tiprack["A1"])

AssertionError: DiTis can only be configured for the last 0 channels

## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [ ]:
plate = lh.deck.get_resource("plate_01")
await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0ASid0006at0&tm1 1 1 0&xp04330 04330 04330 00000&yp1460 1370 1280 0000&th2450te2450lp1931 1931 1931&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1831 1831 1831&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00551 02110&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0ASid0006er00/00


After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [14]:
await lh.dispense(plate["D1:F1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0DSid0007dm2 2 2&tm1 1 1 0&xp04330 04330 04330 00000&yp1190 1100 1010 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 0100 0100&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&zu0032 0032 0032&zr06180 06180 06180&th2450te2450dv01072 00551 02110&ds1200 1200 1200&ss0050 0050 0050&rv000 000 000&ta000 000 000&ba0000 0000 0000&lm0 0 0&dj00zo000 000 000&ll1 1 1&lv1 1 1&de0020 0020 0020&wt00 00 00&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms0010 0010 0010&mh0000 0000 0000&gi000 000 000&gj0gk0
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0DSid0007er00/00


Let's move the liquid back to the original wells.

In [15]:
await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0ASid0008at0&tm1 1 1 0&xp04330 04330 04330 00000&yp1190 1100 1010 0000&th2450te2450lp1931 1931 1931&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1831 1831 1831&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00551 02110&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0ASid0008er00/00
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0DSid0009dm2 2 2&tm1 1 1 0&xp04330 04330 04330 00000&yp1460 1370 1280 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 01

## Dropping tips

Finally, you can drop tips anywhere on the deck by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [16]:
await lh.drop_tips(tiprack["A1:C1"])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0TRid0010xp01629 01629 01629 00000&yp1458 1368 1278 0000&tm1 1 1 0&tt01tp1314tz1414th2450ti0
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0TRid0010er00/00kz381 356 365 000 000 000 000 000vz303 360 368 000 000 000 000 000


In [17]:
await lh.stop()